<a href="https://colab.research.google.com/github/AnilZen/centpy/blob/master/notebooks/Euler_1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shallow Water Equations with CentPy in 1D

### Import packages

In [ ]:
# Install the centpy package
!pip install centpy

In [1]:
# Import numpy and centpy for the solution 
import numpy as np
import centpy

In [2]:
# Imports functions from matplotlib and setup for the animation
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
import os
import csv

### Equation

We solve the shallow water equations in 1D with channel slope and bed-friction source terms 

\begin{equation} 
\partial_t 
\begin{bmatrix} h \\ q \end{bmatrix} 
+ 
\partial_x 
\begin{bmatrix} q \\ (\frac{q^2}{h}+\frac{1}{2}g'h^2) \end{bmatrix} 
= \begin{bmatrix} 0 \\ g'S_oh-\frac{c_f}{2}\frac{q^2}{h^2} \end{bmatrix}  
\end{equation}

with time-varying inlet boundary condition at inlet and outlet (Neumann 0) boundary.

Periodic box.

In [8]:
# problem-specific params
# included in the package in an ugly way, to be fixed in a more elegant way
# but this part should also be kept for BC and initialization
channel_slope = 0.05011
normal_depth = 0.00798
normal_velocity = 1.038
gravity_coeff = 9.81
#froude = 3.71
fc = 2.0*channel_slope/(normal_velocity/(np.sqrt(gravity_coeff*normal_depth)))**2
dist_amp = 0.10
dist_period = 0.933
L_x = 1.30

In [9]:
pars = centpy.Pars1d(x_init=0.0, x_final=L_x, t_final=12.0, dt_out=0.25, J=625, cfl=0.45, scheme="sd2")

In [10]:
# swe
class swe1d(centpy.Equation1d):
    def initial_data(self):
        x = self.x
        u = np.zeros((self.J + 4, 2))
        # midpoint = int(self.J / 2) + 2

#         left_v = [1, 0, 1.0 / (self.gamma - 1.0)]
#         right_v = [0.125, 0.0, 0.1 / (self.gamma - 1.0)]
        # normal_flow = [ normal_depth, normal_depth*normal_velocity ]
        u[:,0] = normal_depth*(1.0+dist_amp*np.sin(2.0*np.pi*x/L_x))
        u[:,1] = normal_velocity*normal_depth
        # u = normal_flow
        return u

    def boundary_conditions(self, u, t):
        u[0,0] = u[-4,0]
        u[0,1] = u[-4,1]
        u[1,0] = u[-3,0]
        u[1,1] = u[-3,1]
        u[-2,0] = u[2,0]
        u[-2,1] = u[2,1]
        u[-1,0] = u[3,0]
        u[-1,1] = u[3,1]


    def flux_x(self, u):
        f = np.zeros_like(u)

        f[:, 0] = u[:, 1]
        f[:, 1] = (np.square(u[:, 1])/u[:, 0]+0.50*gravity_coeff*np.square(u[:, 0]))

        return f

    def spectral_radius_x(self, u):
        q = u[:, 1] 
        vel = q/u[:, 0]
        return 1.0 * np.abs(vel) + 1.0 * np.sqrt(gravity_coeff*u[:, 0])
        # return 1.0*np.abs(u_x)  + 1.0*np.sqrt(self.gamma * p / rho)
        
    def source(self, u, t):
        s = np.zeros_like(u)
        q = u[:, 1] 
        vel = q/u[:, 0]
        s[:, 0] = 0.0
        s[:, 1] = gravity_coeff*channel_slope*u[:,0]-fc/2.0*(vel**2)
        

In [8]:
fc

0.0072816800093554734

### Solution

In [11]:
eqn = swe1d(pars)
soln = centpy.Solver1dSWE(eqn)
soln.solve()

### Animation

In [12]:
# Animation 

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(16,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

# Set the labels
ax1.set_xlabel('x')
ax1.set_ylabel(r'$h$')
ax1.set_xlabel('x')
ax2.set_ylabel(r'$q$')


# Axis limits and lines
line_u=[]
for ax in [ax1, ax2]:
  ax.set_xlim(0.0, L_x)
  ax.set_ylim(0.0, 0.016)
  line_u.append(ax.plot([], [], linewidth=1, color='b', marker='o', markersize=2)[0])

plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
plt.rcParams.update({'font.family': 'Times New Roman','font.size':19})

# animation function.  This is called sequentially
j0 = slice(2,-2)
def animate(i):
    h = soln.u_n[i,j0,0]
    q = soln.u_n[i,j0,1]
    line_u[0].set_data(soln.x[j0], h)
    line_u[1].set_data(soln.x[j0], q)

plt.close()
anim = animation.FuncAnimation(fig, animate, frames=soln.Nt, interval=100, blit=False);
HTML(anim.to_html5_video())


In [13]:
np.shape(soln.u_n)

(49, 629, 2)

In [14]:
# text files output
un_shape = np.shape(soln.u_n)
frames = un_shape[0]
j0 = slice(2,-2)
for i in range(0, frames):
    t = 0.25*i
    format_string_time = f"{t:.1f}"
    file_name = 'outXYZ_%s' % format_string_time
    with open(file_name, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(np.transpose(soln.x[j0]),np.transpose(soln.u_n[i,j0,0])))